In [ ]:
#default_exp models.ensemble

# Ensemble models

Classes for ensembling several `Learner`s into one model

In [ ]:
# export
from fastai.basics import *
from fastai.tabular.all import *

In [ ]:
#export
class Ensemble():
    def __init__(self, dls, n_models:int=10, learn_cls=Learner, **learner_kwargs):
        "Create an ensemble of `Learner`s. learner_func is "
        
        self.dls = dls
        self.metrics = learner_kwargs['metrics']
        self.models = []
        for _ in range(n_models):
            self.models.append(learn_cls(dls=dls, **learner_kwargs))
        
    def fit_one_cycle(self, n_iterations, max_lr, **kwargs):
        "Fit the models with fit_one_cycle"
        for m in self.models:
            m.fit_one_cycle(n_iterations, max_lr=max_lr, **kwargs)
            
    def validate(self, dl=None) -> pd.DataFrame:
        "Validate all models individually and as an ensemble"
        if dl is None: dl=self.dls[1]
        model_results = torch.cat([m.get_preds(reorder=False, dl=dl)[0] for m in self.models], dim=-1)
        ensemble_results = model_results.sum(axis=-1) / len(self.models)
        res_df = pd.DataFrame(columns=['model_identifier'] + [m.name if hasattr(m, 'name') else m.__name__ for m in self.metrics])
        res_df.loc[0] = (['ensemble'] 
                         + [metric(ensemble_results, Tensor(dl.y.values)).item() for metric in self.metrics])
        for i in range(len(self.models)):
            res_df.loc[i+1] = ([i] 
                               + [metric(model_results[:,i], Tensor(dl.y.values)).item()  for metric in self.metrics])
        return res_df
    
    def get_ensemble_preds(self, ds_idx=1, dl=None, with_input=True, with_decoded=False, with_loss=False, act=None,
                           inner=False, reorder=False, cbs=None, **kwargs):
        "get_preds but ensemble results"
        if dl is None: dl=self.dls[1].new(shuffled=False, drop_last=False)
        if reorder and hasattr(dl, 'get_idxs'):
            idxs = dl.get_idxs()
            dl = dl.new(get_idxs = _ConstantFunc(idxs))
        model_results = []
        for m in self.models:
            model_results.append(m.get_preds(dl=dl, with_input=with_input, with_decoded=with_decoded, with_loss=with_loss,
                                             act=act, inner=inner, reorder=reorder, cbs=cbs, **kwargs))
        
        ensemble_results = []
        # iterate through results, could work better:
        obs_idx = 0
        if with_input: 
            ensemble_results.append(model_results[0][obs_idx])
            obs_idx += 1
        ensemble_results.append(sum([res[obs_idx] for res in model_results])/len(self.models))
        obs_idx += 1
        ensemble_results.append(model_results[0][obs_idx])
        obs_idx += 1
        if with_decoded:
            ensemble_results.append(model_results[0][obs_idx])
            obs_idx += 1
        if with_loss:
            ensemble_results.append(sum([res[obs_idx] for res in model_results])/len(self.models))
        return tuple(ensemble_results)
    
    def predict(self, item):
        model_results = [m.predict(item) for m in self.models]
        ensemble_results = sum([res[-1] for res in model_results])/len(self.models)
        ensemble_dec_results = sum([res[-2] for res in model_results])/len(self.models)
        return (model_results[0][0], ensemble_dec_results, ensemble_results)

Create `Ensemble` of `Learners` with `n_models`. Works pretty much like a single `Learner`

In [ ]:
# export

@delegates(Ensemble.__init__)
def tabular_ensemble(dls, n_models:int=10, layers=None, emb_szs=None, config=None, n_out=None, y_range=None, **kwargs):
    "Function to create Ensemble containing `TabularLearner`s"
    if config is None: config=tabular_config()
    if layers is None: layers = [200, 100]
    to = dls.train_ds
    emb_szs = get_emb_sz(dls.train_ds, {} if emb_szs is None else emb_szs)
    if n_out is None: n_out = get_c(dls)
    assert n_out, "`n_out` is not defined and could not be inferred from data, set `dls.c` or pass `n_out`"
    if y_range is None and 'y_range' in config: y_range = config.pop('y_range')
    model = TabularModel(emb_szs, len(dls.cont_names), n_out, layers, y_range=y_range, **config)
    return Ensemble(dls, n_models=n_models, learn_cls=TabularLearner, model=model, **kwargs)